In [20]:
%pip install datasets
%pip install transformers
%pip install peft
%pip install evaluate
%pip install scikit-learn



from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)
from turtle import pd
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np
import pandas as pd

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: peft in c:\users\zhuocheng\appdata\local\programs\python\python312\lib\site-packages (0.10.0)




[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     --------------------------- ------------ 41.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 60.6/60.6 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB 660.6 kB/s eta 0:00:16
   ---------------------------------------- 0.1/10.6 MB 1.3 MB/s eta 0:00:08
   ---------------------------------------- 0.1/10.6 MB 1.2 MB/s eta 0:00:09
    ------------------------------------


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
# load dataset
test_data= pd.read_csv("VUA_test_all.csv", engine="python")
train_data= pd.read_csv("VUA_train.csv", engine="python")
test_data

,id,sentence,word,label
0,a1j-fragment33_494_1,The Labour Party Conference : Policy review th...,The,0
1,a1j-fragment33_494_2,The Labour Party Conference : Policy review th...,Labour,0
2,a1j-fragment33_494_3,The Labour Party Conference : Policy review th...,Party,0
3,a1j-fragment33_494_4,The Labour Party Conference : Policy review th...,Conference,0
4,a1j-fragment33_494_5,The Labour Party Conference : Policy review th...,:,0
...,...,...,...,...
58354,kbw-fragment09_4012_1,Okay ?,Okay,0
58355,kbw-fragment09_4012_2,Okay ?,?,0
58356,kbw-fragment09_4013_1,french fries .,french,0
58357,kbw-fragment09_4013_2,french fries .,fries,0


In [10]:
model_checkpoint = 'distilbert-base-uncased'
# define label maps
id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative":0, "Positive":1}

# generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

c:\Users\zhuocheng\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\zhuocheng\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of DistilBertForSequenceClassification were not initialized from t

In [14]:
# create tokenize function

# create tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

# add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))


def tokenize_function(examples):
    # extract text
    text = examples["sentence"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

# tokenize training and validation datasets
# tokenize text in the DataFrame
tokenized_data = train_data.apply(tokenize_function, axis=1, result_type='expand')

# merge the tokenized data with the original DataFrame
data = pd.concat([train_data, tokenized_data], axis=1)

# print the first few rows of the tokenized data
print(data.head())

                   id                                           sentence  \
0  a1e-fragment01_1_1  Latest corporate unbundler reveals laid-back a...   
1  a1e-fragment01_1_2  Latest corporate unbundler reveals laid-back a...   
2  a1e-fragment01_1_3  Latest corporate unbundler reveals laid-back a...   
3  a1e-fragment01_1_4  Latest corporate unbundler reveals laid-back a...   
4  a1e-fragment01_1_5  Latest corporate unbundler reveals laid-back a...   

        word  label          0               1  
0     Latest      0  input_ids  attention_mask  
1  corporate      0  input_ids  attention_mask  
2  unbundler      0  input_ids  attention_mask  
3    reveals      1  input_ids  attention_mask  
4  laid-back      0  input_ids  attention_mask  


In [16]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [21]:
# import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

# define an evaluation function to pass into trainer later
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

In [23]:
# define list of examples
text_list = ["The Labour Party Conference : Policy review throws a spanner in the Whitehall machinery", "Not a fan, don't recommed.", "Better than the first one.", "This is not worth watching even once.", "This one is a pass."]

print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
    # compute logits
    logits = model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits)

    print(text + " - " + id2label[predictions.tolist()])

Untrained model predictions:
----------------------------
The Labour Party Conference : Policy review throws a spanner in the Whitehall machinery - Negative
Not a fan, don't recommed. - Negative
Better than the first one. - Negative
This is not worth watching even once. - Negative
This one is a pass. - Negative
